In [26]:
# Report V1. Sum total of all aWattar prices over periods from 00:00 to 24:00=00:00 …. V1 = Sum (Awattar) - DONE
# Report V2. Calculation of average aWattar prices of this period (T) in hours (days * 24) … V2 = V1/T - DONE
# Report V3. Calculation of energy consumed over period T … V3 = Sum (Energy) - DONE
# Report 4. Calculation of aWattar reference costs … Cref = V2 x V3 / 1000 - DONE

# Report 5. Calculation of costs for every period of energy of hour x awattar of hour … c = e * a /1000
# Report 6. Calculation of sum of hourly energy costs … C = Sum (c)
# Report 7. Calculation of average savings in € … S€ = Cref – C
# Report 8. Calculation of average savings in % … S% = (Cref – C) / Cref *100

import sys 
sys.path.append('/home/pi/smart_plug/')
import requests
from datetime import datetime, timedelta
import pytz
import pandas as pd
import json
import math
import os
from main_services.common_utils import common_utils 

auth_url = common_utils.static_smart_meter_service_link+'orchestration/Authentication/Login'
auth_payload = {"user": common_utils.static_smart_meter_username, "pwd": common_utils.static_smart_meter_password}
auth_response = None

# Report V1 
def sumAwattarPricesByDates_V1V2():
    awattar_json_url = "https://api.awattar.at/v1/marketdata?start={}&end={}"

    timezone = 'Europe/Vienna'  # Replace with your desired timezone
    # Get the timezone object
    desired_timezone = pytz.timezone(timezone)

    # Get the current date and time in the desired timezone
    current_datetime = datetime.now(desired_timezone)

    # Calculate the date 7 days ago
    seven_days_ago = current_datetime - timedelta(days=7)

    # Set the time to the start of the day
    start_of_day = datetime(current_datetime.year, current_datetime.month, current_datetime.day, 0, 0, 0, 0, desired_timezone)
    start_of_day_seven_days_ago = datetime(seven_days_ago.year, seven_days_ago.month, seven_days_ago.day, 0, 0, 0, 0, desired_timezone)

    # Set the time to the end of the day
    end_of_day = datetime(current_datetime.year, current_datetime.month, current_datetime.day, 23, 59, 59, 999000, desired_timezone)
    end_of_day_seven_days_ago = datetime(seven_days_ago.year, seven_days_ago.month, seven_days_ago.day, 23, 59, 59, 999000, desired_timezone)

    # Convert the datetime objects to timestamps
    timestamp_start_of_day_seven_days_ago = int(start_of_day_seven_days_ago.timestamp() * 1000)
    timestamp_end_of_day_seven_days_ago = int(end_of_day_seven_days_ago.timestamp() * 1000)
    timestamp_start_of_day_current_datetime = int(start_of_day.timestamp() * 1000)
    timestamp_end_of_day_current_datetime = int(end_of_day.timestamp() * 1000)

    start_of_day, end_of_day, number_of_date = get_start_and_end_of_day(timestamp_start_of_day_seven_days_ago, timestamp_end_of_day_current_datetime, timezone)

    ## print(f"Awattar After Start of the day: {start_of_day}")
    ## print(f"Awattar After End of the day: {end_of_day}")

    # Get Awattar Data
    json_url = awattar_json_url.format(start_of_day, end_of_day)
    
    awattar_json_response = requests.get(json_url).json()
    awattar_json_response = pd.json_normalize(awattar_json_response['data'])
    awattar_json_response['start_timestamp'] = pd.to_datetime(awattar_json_response['start_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    awattar_json_response['end_timestamp'] = pd.to_datetime(awattar_json_response['end_timestamp'], unit='ms') + pd.Timedelta(hours=1)
    ## print(awattar_json_response)

    # Calculate the sum of marketprice
    total_marketprice = awattar_json_response['marketprice'].sum()

    print("Report - V1:", roundUpCeil(total_marketprice))

    T = number_of_date * 24

    print("Report - V2:", roundUpCeil(total_marketprice/T))

    report1 = roundUpCeil(total_marketprice)
    report2 = roundUpCeil(total_marketprice/T)

    return report1, report2

# Report V3
def sumEnegrySmartMeterByDates_V3(): 
    ## SMART METER STEP1: Login into smart-meter
    auth_url = common_utils.static_smart_meter_service_link+'orchestration/Authentication/Login'
    auth_payload = {"user": common_utils.static_smart_meter_username, "pwd": common_utils.static_smart_meter_password}
    auth_response = None

    auth_cookie, auth_xsrf_token, nsc_wt = None, None, None
    auth_response = requests.post(auth_url, json=auth_payload)

    if auth_response.status_code == 200:
        print("The status code is 200 (OK).")
        print("Smart meter - Authentication successful...")
        auth_cookie = auth_response.cookies['__Host-go4DavidSecurityToken']
        auth_xsrf_token = auth_response.cookies['XSRF-Token']
        nsc_wt = auth_response.cookies['NSC_WT_TWYUXFCQ-TTM']
    else:
        raise Exception("Authentication failed with status code: getSmartMeterDataFromYesterday",auth_response.status_code)

    headers = {
        'Cookie': f'__Host-go4DavidSecurityToken={auth_cookie};XSRF-Token={auth_xsrf_token};NSC_WT_TWYUXFCQ-TTM={nsc_wt};',
    }

    # Replace with your desired timezone
    timezone = 'Europe/Vienna'

    # Set the timezone
    tz = pytz.timezone(timezone)

    # Get today's date with the specified timezone
    today_date = datetime.now(tz).strftime("%Y-%m-%d")

    # Calculate 7 days before today with the specified timezone
    seven_days_before = datetime.now(tz) - timedelta(days=7)
    seven_days_before_date = seven_days_before.strftime("%Y-%m-%d")

    # Print the dates for verification
    # print("7 Days Before:", seven_days_before_date)
    # print("Today's Date:", today_date)

    ## SMART METER STEP2: Get data from smart-meter reading
    data_url = f"{common_utils.static_smart_meter_service_link}orchestration/ConsumptionRecord/Week?meterId={common_utils.static_smart_meter_meter_id}&startDate={seven_days_before_date}&endDate={today_date}"
    data_response = requests.get(data_url, headers=headers)
    outputData = None
    # print("SMART METER :", data_url)

    if data_response.status_code == 200:
        outputData = pd.DataFrame(json.loads(data_response.content))[['meteredValues', 'peakDemandTimes']]
        # Convert 'peakDemandTimes' to datetime and add 1 hour
        # outputData['peakDemandTimes'] = pd.to_datetime(outputData['peakDemandTimes']) + pd.Timedelta(hours=1)
        # print("Output Data From SmartMeter:", outputData)
        # print("Sum of MeteredValues:", outputData['meteredValues'].sum())
        # print("Round sum of MeteredValues:", round(outputData['meteredValues'].sum(), 2))

    else:
        print("Data request failed with status code:", 'DATA FAILURE')
        raise Exception("Data request failed with status code: Smartmeter_service2", data_response.status_code)

    # Logout the smart meter service
    # https://smartmeter.netz-noe.at/orchestration/Authentication/Logout
    auth_logout_url = common_utils.static_smart_meter_service_link+'orchestration/Authentication/Logout'
    auth_response = requests.get(auth_logout_url, headers=headers)
    if auth_response.status_code == 200:
        print("Logout success.")
    else:
        print("Logout failed.")


    if len(outputData) > 0:
        # print("Data fetch success.")
        sumOfSmartMeterData = round(outputData['meteredValues'].sum(), 2)
        print("Report - V3:", sumOfSmartMeterData)
        return sumOfSmartMeterData
    else:
        raise Exception("Data request failed with status code: Smartmeter_service2", 404)

# Report V4
def awattarReferenceCost_CREF(): 
    # Cref = V2 x V3 / 1000
    V2 = sumAwattarPricesByDates_V1V2()[1]
    V3 = sumEnegrySmartMeterByDates_V3()
    Cref = roundUpCeil(V2 * V3 / 1000)
    print("Report - V4:", Cref)

# Report V5





"""
Other utility methods to calculate the reports
21.01.24
"""

def get_start_and_end_of_day(from_timestamp, to_timestamp, timezone='CET'):
    # Convert timestamps to datetime objects
    from_date = datetime.utcfromtimestamp(from_timestamp / 1000)
    to_date = datetime.utcfromtimestamp(to_timestamp / 1000)

    # Set the timezone to UTC
    from_date_utc = pytz.timezone('UTC').localize(from_date)
    to_date_utc = pytz.timezone('UTC').localize(to_date)

    # Convert UTC to the specified local time zone
    from_date_local = from_date_utc.astimezone(pytz.timezone(timezone))
    to_date_local = to_date_utc.astimezone(pytz.timezone(timezone))

    # Get the start and end of the day for both from and to dates
    start_of_day_from = from_date_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day_from = start_of_day_from.replace(hour=23, minute=59, second=59, microsecond=999999)

    start_of_day_to = to_date_local.replace(hour=0, minute=0, second=0, microsecond=0)
    end_of_day_to = start_of_day_to.replace(hour=23, minute=59, second=59, microsecond=999999)

    # Get the start of the next day for both from and to dates
    start_of_next_day_from = start_of_day_from + timedelta(days=1)
    start_of_next_day_to = start_of_day_to + timedelta(days=1)

    # Convert datetime objects to Unix timestamps in milliseconds
    start_timestamp_from = int(start_of_day_from.timestamp()) * 1000
    end_timestamp_from = int(start_of_next_day_from.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    start_timestamp_to = int(start_of_day_to.timestamp()) * 1000
    end_timestamp_to = int(start_of_next_day_to.timestamp()) * 1000 - 1  # Subtract 1 millisecond

    ##print(f"Start of the day for 'From': {start_of_day_from}")
    ##print(f"End of the day for 'From' (including next start): {end_of_day_from}")

    ##print(f"Start of the day for 'To': {start_of_day_to}")
    ##print(f"End of the day for 'To' (including next start): {end_of_day_to}")
    
    number_of_date = numberOfDaysFromDates(start_timestamp_from, end_timestamp_to)

    return start_timestamp_from, end_timestamp_to, number_of_date

def numberOfDaysFromDates(start_timestamp_from, end_timestamp_to):
    
    # Convert timestamps to datetime objects
    start_date = datetime.utcfromtimestamp(start_timestamp_from / 1000)
    end_date = datetime.utcfromtimestamp(end_timestamp_to / 1000)

    # Calculate the difference in days
    days_difference = (end_date - start_date).days

    ##print(f"Number of days between the two dates: {days_difference} days")
    return days_difference

def roundUpCeil(value):
    return math.ceil(value)


## Test all methods
# sumAwattarPricesByDates_V1V2()
# sumEnegrySmartMeterByDates_V3()
awattarReferenceCost_CREF()





Report - V1: 18425
Report - V2: 96
The status code is 200 (OK).
Smart meter - Authentication successful...
Logout success.
Report - V3: 365.94
Report - V4: 36
